# Название и описание проекта

**Цель исследования —**

# Содержание проекта:

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#Ссылка 1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Заголовок 1</a></span></li>
        <li><span><a href="#Ссылка 2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Заголовок 2</a></span></li>
        <li><span><a href="#Ссылка 3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Заголовок 3</a></span></li>
    </ul>
</div>

Импортируем полезные библиотеки, объявим константы и зададим параметры по умолчанию

In [ ]:
# data manipulation
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# math
import math
import scipy.stats as st

# utility
from IPython.core.display import display, HTML
from tqdm import tqdm

# time series
import calendar
import datetime

# sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, roc_curve, roc_auc_score, 
from sklearn.metrics import mean_squared_error, root_mean_squared_error,r2_score
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# lightgbm
from lightgbm import LGBMRegressor

# neural networks
import torch
import torch.nn as nn

# constant values
RANDOM_STATE = np.random.RandomState(884002)
SMALL_SIZE = 12
MEDIUM_SIZE = 18
BIGGER_SIZE = 24

# set default values
display(HTML("<style>.container { width:75% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.4f}'.format)
plt.rc('font', size=SMALL_SIZE)                                    # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)                              # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)                              # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)                              # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)                              # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)                              # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)                            # fontsize of the figure title
plt.rc('figure', figsize=(18, 12))                                 # controls figure size
sns.set(rc={'figure.figsize':(18, 12)})

## Загрузка и разведочный анализ данных

### Получим данные безопасным способом при помощи конструкции try-except:

In [ ]:
try:
    data = pd.read_csv('/datasets/real_estate_data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/real_estate_data.csv')

### Ознакомимся с набором данных. Выведем несколько строк из датафрейма, взятых случайным образом:

In [ ]:
data.sample(random_state=RANDOM_STATE)

### Отобразим информацию для краткого обзора данных:

In [ ]:
data.info()

### Отобразим таблицу с описательной статистикой столбцов:

In [ ]:
data.describe()

### Отобразим таблицу с попарными корреляциями столбцов:

In [ ]:
data.corr()

### Отобразим гистограммы распределений значений столбцов:

In [ ]:
fig = plt.figure()
plt.subplots_adjust(wspace=0.25, hspace=0.8)

for i, col in enumerate(data.columns.difference([''])):
    ax = fig.add_subplot(4, 3, i + 1)
    
    # Отобразим количественные и категориальные переменные соответствующими методами
    if np.issubdtype(data[col].dtype, np.number):
        data[col].plot(kind='hist')
    else:
        data[col].value_counts().plot(kind='bar')
    
    ax.set_title(col)
    ax.grid(visible=True)

### Построим гисограммы для всех количественных столбцов:

In [ ]:
data.hist(figsize=(15, 20));

### Краткий вывод:
В данных были обнаружены следующие проблемы (или их отсутствие):
- в данных присутствуют/отсутствуют нарушения правила хорошего стиля в названиях столбцов
- в данных присутствуют/отсутствуют несоответствия типов в столбцах ``
- в данных присутствуют/отсутствуют намёки на мультиколлинеарность

## Предобработка данных

In [ ]:
# check
data.columns

### Исправим нарушения правил хорошего стиля в названиях столбцов:

* несколько слов в названии запишем в «змеином_регистре»
* все символы сделаем строчными
* устраним пробелы

In [ ]:
data.rename(inplace=True, columns={})

### Исправим несоответствия типов в столбцах:

In [ ]:
data = data.astype({})

# check
data.dtypes

### Добавим новые столбцы:

Категоризуем данные

### Удалим неинформативные столбцы, которые не несут ценности для прогноза::

In [ ]:
data.drop([''], axis=1, inplace=True)

# check
data.columns

### Обработаем пропущенные значения:

Сначала посчитаем, сколько в таблице пропущенных значений

In [ ]:
data.isna().sum()

Различают следующие 3 механизма формирования пропусков:
1. **MCAR**. Механизм формирования пропусков, при котором вероятность пропуска для каждой записи набора одинакова. Столбцы, которые имеют такой механизм формирования пропусков:
    - ` `
2. **MAR**. Механизм формирования пропусков, при котором вероятность пропуска может быть определена на основе другой имеющейся в наборе данных информации, не содержащей пропуски. Столбцы, которые имеют такой механизм формирования пропусков:
    - ` `
3. **MNAR**. Механизм формирования пропусков, при котором вероятность пропуска могла бы быть описана на основе других атрибутов, но информация по этим атрибутам в наборе данных отсутствует (например, объект недвижимости может иметь подземный этаж или чердак, но такая информация в наборе данных отсутствует). Столбцы, которые имеют такой механизм формирования пропусков:  
    - ` `

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения

In [ ]:
data.isna().sum()

### Обработаем дубликаты:

Сначала обработаем неявные дубликаты

Выведем на экран количество полных строк-дубликатов

In [ ]:
data.duplicated().sum()

Удалим полные дубликаты из таблицы

In [ ]:
data = data.drop_duplicates()

Выведем на экран неполные дубликаты по подмножеству стобцов

In [ ]:
data[data.duplicated(subset=[])]

Удалим неполные дубликаты из таблицы

In [ ]:
data = data.drop_duplicates(subset=[])

Ещё раз посчитаем явные дубликаты в таблице и убедимся, что полностью от них избавились

In [ ]:
data.duplicated().sum()

### Обработаем аномальные значения:

Учтем особенности фильтрации `pandas`, чтобы не потерять записи с пропусками

### Краткий вывод:
В данных были устранены следующие проблемы:
- нарушения правил хорошего стиля в названиях столбцов
- несоответствия типов в столбцах
- неинформативные признаки, которые не несут ценности для прогноза
- пропуски в столбцах ``
- явные и неявные дубликаты
- аномальные значения в столбцах ``

## Исследовательский анализ данных:

### Изучим баланс классов:

### Проанализируем как целевой признак `` связан со всеми остальными признаками:

In [ ]:
corr_data = data.corr()[''].sort_values(key=abs)
abs(corr_data).plot(kind='barh', 
                    fontsize=14,
                    color=(corr_data > 0).map({True: 'g', False: 'r'}),
                    edgecolor='black',
                    title='',
                    grid=True);

### Краткий вывод:

## Обучение моделей:

### Разделим исходные данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=12345, stratify=data.is_ultra)
train_data, valid_data = train_test_split(train_data, test_size=0.25, random_state=12345, stratify=train_data.is_ultra)

split = pd.DataFrame(columns=['proprotion'])
for dataset in [train_data, valid_data, test_data]:
    split[dataset] = len(dataset) / len(data)

In [ ]:
# Разделим обучающую выборку
train_features = train_data.drop(['is_ultra'], axis=1) 
train_target = train_data['is_ultra']

# Разделим валидационную выборку
valid_features = valid_data.drop(['is_ultra'], axis=1)
valid_target = valid_data['is_ultra']

# Разделим тестовую выборку
test_features = test_data.drop(['is_ultra'], axis=1)
test_target = test_data['is_ultra']

### Создадим базовый пайплайн для предобработки данных:

### Создадим модели:

### Кросс-валидация:

### Краткий вывод:

## Тестирование и анализ наилучшей модели:

In [ ]:
probabilities_one_valid = forest_model.predict_proba(valid_features)[:, 1]
fpr, tpr, thresholds = roc_curve(valid_target, probabilities_one_valid)

plt.figure()
# ROC-кривая модели RandomForestClassifier
plt.plot(fpr, tpr)

# ROC-кривая случайной модели (выглядит как прямая)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim(0, 1.0)
plt.ylim(0, 1.0)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()

print('Площадь под ROC-кривой (AUC-ROC) равна:', roc_auc_score(valid_target, probabilities_one_valid))

### Краткий вывод:

## Общий вывод: